# 1. Setup

## 1.1 Module import

Import necessary modules

In [ ]:
import tensorflow as tf
import keras
import dataset_utils

## 1.2 Dataset import using opencv

Define parameters for the image import

In [ ]:
img_width = 100
img_height = 40

train_dataset_path = r'/mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/training_dataset/all_cropped'
validation_dataset_path = r'/mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/validation_dataset/all_cropped'

Import the data for the training dataset

In [ ]:
train_labels, train_images = dataset_utils.load_dataset(path_to_dataset = train_dataset_path, change_contrast = True, grayscale=True, image_size = (img_width, img_height))

Import the data for the validation dataset

In [ ]:
validation_labels, validation_images = dataset_utils.load_dataset(path_to_dataset = validation_dataset_path, change_contrast = True, grayscale=True, image_size = (img_width, img_height))

In [ ]:
print(train_images[0][:,:,1])

In [ ]:
import numpy as np

reshaped_train_images = []
reshaped_val_images = []

shape = train_images[0].shape

for train_image_index in range(len(train_images)):
    reshaped_train_images.append(np.reshape(train_images[train_image_index][:,:,1], (shape[0], shape[1], 1)))

for validation_image_index in range(len(validation_images)):
    reshaped_val_images.append(np.reshape(validation_images[validation_image_index][:,:,1], (shape[0], shape[1], 1)))

reshaped_train_images = np.array(reshaped_train_images)
reshaped_val_images = np.array(reshaped_val_images)

# 2. Transfer learning

## 2.1 Construct the new model

Load the base model

In [ ]:
IMG_SIZE = (img_height, img_width)
IMG_SHAPE = IMG_SIZE + (1,)

X_input = keras.layers.Input(IMG_SHAPE)

# CONV -> BN -> RELU Block applied to X
X = keras.layers.Conv2D(16, (3, 3), strides=(1, 1), name='conv0', padding='same')(X_input)
X = keras.layers.MaxPooling2D((2, 2))(X)
X = keras.layers.Conv2D(8, (5, 5), strides=(1, 1), name="conv2", padding='same')(X)
X = keras.layers.BatchNormalization(axis=3, name='bn0')(X)
X = keras.layers.Activation('relu')(X)

# MAXPOOL
X = keras.layers.MaxPooling2D((2, 2), name='max_pool')(X)

# FLATTEN X (means convert it to a vector) + FULLYCONNECTED
X = keras.layers.Flatten()(X)
X = keras.layers.Dense(64, activation="selu")(X)
X = keras.layers.Dense(3, activation='sigmoid', name='fc')(X)

# Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
model = keras.models.Model(inputs=X_input, outputs=X, name='HappyModel')

Add the custom classification layer to it

## 2.2 Start the initial learning process

Define Parameters for the learning process

In [ ]:
base_learning_rate = 0.03
initial_epochs = 60

Compile the before built model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

Train the model!

In [ ]:
generator = keras.preprocessing.image.ImageDataGenerator()
batch_size = 32

history = model.fit(generator.flow(reshaped_train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=(reshaped_val_images, validation_labels), epochs=initial_epochs, callbacks=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001))

In [ ]:
model.evaluate(x = validation_images, y = validation_labels)

# 4. Save the model

Info: Loading can be done via: ***model = tf.keras.models.load_model(PATH_TO_MODEL_FILE)***


In [ ]:
# for now until it is implemented properly: manually make sure that the path exists (in this case '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/')
save_path = '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/happy_house_w100_h40_no_contrast_grayscale_train_99_val_97.keras'

model.save(save_path)